# Unify rasters to a base raster

In [1]:
# Standard libraries
import rasterio
import os
from importlib.resources import files
from rasterio.enums import Resampling
from pathlib import Path
from tqdm import tqdm

# Custom modules
from beak.utilities.io import save_raster, create_file_folder_list, create_file_list, check_path
from beak.utilities.raster_processing import unify_raster_grids

In [2]:
BASE_PATH = files("beak.data")

BASE_NAME = "GEOPHYSICS_AERORADIOMETRICS"
BASE_SPATIAL = "EPSG_4326_RES_0_015"
BASE_EXTENT = "CLIPPED_USC"

input_folder = BASE_PATH / BASE_NAME / "RAW" / "USGS"
output_folder = BASE_PATH / BASE_NAME / "PROCESSED" / BASE_SPATIAL / BASE_EXTENT / "USGS" / "UNIFIED"
base_raster = BASE_PATH / "BASE_RASTERS" / str(BASE_SPATIAL + "_" + BASE_EXTENT + ".tif")


In [3]:
# Create file and folder list
folders, _ = create_file_folder_list(input_folder)
folders.insert(0, input_folder)

file_list = []
for folder in folders:
  folder_files = create_file_list(folder, recursive=True)
  file_list.extend(folder_files)
  
print(f"Found {len(file_list)} files in {len(folders)} folders:")

for file in file_list:
  print(file)

Found 3 files in 1 folders:
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\GEOPHYSICS_AERORADIOMETRICS\RAW\USGS\NAMrad_K.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\GEOPHYSICS_AERORADIOMETRICS\RAW\USGS\NAMrad_Th.tif
s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\GEOPHYSICS_AERORADIOMETRICS\RAW\USGS\NAMrad_U.tif


In [9]:
dry_run = False
if dry_run:
  print("Dry run, no files will be written.\n")
  
for file in tqdm(file_list, total=len(file_list)):
  out_path = output_folder / file.relative_to(input_folder)
  check_path(Path(os.path.dirname(out_path)))
  
  raster = rasterio.open(file)
  unified_raster, meta = unify_raster_grids(base_raster, [file], resampling_method=Resampling.bilinear, same_extent=True, same_shape=True)[0]

  if not dry_run:
    save_raster(out_path,
          array=unified_raster,
          crs=meta['crs'],
          height=meta['height'],
          width=meta['width'],
          nodata_value=meta['nodata'],
          transform=meta['transform'],
          dtype="float32")


100%|██████████| 3/3 [00:03<00:00,  1.03s/it]
